In [3]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import pandas as pd
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import matplotlib as mpl
mpl.rc("font", family='Droid Sans Fallback', weight="bold")
import matplotlib.pyplot as plt
start, end = '20170104', '20230418'

当同时满足如下两个条件时会出发买入操作：

1、相对价格连续 12天以上（含 12 天）在 MA10 之下运行     
2、相对价格收盘价距离 MA10 有一定空间，以触发条件时当天收盘价买入。

在卖出条件方面，只要符合如下三个条件其中之一则会引发卖出操作：

1、持有天数到达阈值     
2、持有期间内相对价格回抽MA10          
3、持有期间内发生较大幅度的亏损引发止损操作，以满足卖出条件时当天收盘价卖出。

标准：相对zz1000的价格

In [7]:
close_all = read_data('close')
zz1000 = pd.read_pickle('/mydata2/wangs/data/feature/zz1000.pk').loc[start:end,'close']

# 需要的数据：
1、相对价格与MA10之间的关系           
2、收盘价    
# 可以优化的数据
1、连续低于MA10的天数 N      
2、据MA10的空间（(MA10 - re_close)/abs(re_close))      
3、止盈止损程度        
4、持有天数      

In [52]:
def read_data(name,start,end):
    if name in ('open', 'close', 'high', 'low'):
        result = (ff.read(name) * ff.read('post') * ff.filter0).loc[:, start:end]
    else:
        result = (ff.read(name) * ff.filter0).loc[:, start:end]
    return result

def get_data(start,end):
    _close_all = read_data('close',start,end)
    _zz1000 = pd.read_pickle('/mydata2/wangs/data/feature/zz1000.pk').loc[start:end,'close']
    # _ret = ff.rets_all.loc[_close_all.index,_close_all.columns]
    _re_close_all = _close_all - _zz1000
    _MA10 = _re_close_all.rolling(window = 10,min_periods = 1 ,axis =1).mean()
    _relat = (_re_close_all - _MA10)/abs(_re_close_all) # 符号代表关系，大小代表距离
    _index_list = _zz1000.index
    _close_values = _close_all.values
    _relat_values = _relat.values
    _stock_lst = _close_all.index
    return _close_values,_relat_values,_index_list,_stock_lst

def Strategy_sub(para):
    _close,_relate,_N1,_lamb,_slp,_N2 = para
    _hold = [0] * len(_close)
    k = 0 #持仓状态
    for i in range(len(_close)):
        _hold[i] = k
        if k == 0:
            if _relate[i] < 0:
                j += 1
            else:
                j = 0
            if (j >= _N1) & (_relate[i] <= _lamb):
                k = 1
                N = 0 # 持有天数
                first_close = _close[i]  
        else:
            j = 0
            N += 1
            if (_relate[i] >= 0) or (N >= _N2) or ((_close[i]/first_close - 1) < _slp):
                k = 0
                _hold[i] = 0
                N = 0
    return _hold

class relative_price_stragegy(object):
    def __init__(self,start,end):
        self.close,self.relate,self.date_lst,self.stock_lst = get_data(start,end)
    def Strategy(self,N1,lamb,slp,N2):
        # N1 连续低于MA10的天数,lamb 距MA10的空间,slp 止损,N2 最长持有天数
        para_lst = [(self.close[i,:],self.relate[i,:],N1,lamb,slp,N2) for i in range(len(self.stock_lst))]
        with Pool(24) as p:
            res_lst = list(p.imap(Strategy_sub,para_lst))
        hold_df = pd.DataFrame(res_lst,columns = self.date_lst, index = self.stock_lst)
        return hold_df
    def calcu_ret(self,N1,lamb,slp,N2):
        hold_df = self.Strategy(N1,lamb,slp,N2)
        ret_df = ff.rets_all
        hold_ret = (hold_df*ret_df).loc[self.stock_lst,self.date_lst].mean().fillna(0)
        res = ff.cal_returns(hold_ret)
        return hold_ret,res

In [53]:
import optuna as opt
# 一个示例(需要限制开仓率或者限制事件数量)
# 我希望有至少1000的触发次数，80%的成功率和70%的正收益率
example1 = relative_price_stragegy(start,end)
def eve_objective(trial):
    N1 = trial.suggest_int('N1', 5, 25)
    lamb = trial.suggest_uniform('lamb', -0.5, -0.01)
    slp = trial.suggest_uniform('slp', -0.3, -0.01)
    N2 = trial.suggest_int('N2', 5, 30)   
    hold_ret,res = example1.calcu_ret(N1,lamb,slp,N2)
    return res['夏普率']
study = opt.create_study(direction='maximize')
with tqdm(total=500) as pbar:
    def callback(study, trial):
        pbar.update(1)

    study.optimize(eve_objective, n_trials=500, callbacks=[callback])
best_params = study.best_params
print(study.best_value)

[I 2024-05-09 17:00:07,115] A new study created in memory with name: no-name-3e7e67fc-7e02-48e1-9e73-1e4878cfda6c
  3%|███▌                                                                                                                 | 15/500 [01:21<47:42,  5.90s/it][W 2024-05-09 17:01:34,940] Trial 15 failed with parameters: {'N1': 24, 'lamb': -0.41497029990823553, 'slp': -0.06355464675675505, 'N2': 13.397532035590977} because of the following error: The value nan is not acceptable.
[W 2024-05-09 17:01:34,943] Trial 15 failed with value nan.
  4%|████▍                                                                                                                | 19/500 [01:45<47:53,  5.97s/it][W 2024-05-09 17:01:58,949] Trial 19 failed with parameters: {'N1': 21, 'lamb': -0.4996672542445692, 'slp': -0.054377709369320085, 'N2': 14.294722153348763} because of the following error: The value nan is not acceptable.
[W 2024-05-09 17:01:58,952] Trial 19 failed with value nan.
  4%|████▋  

3.585685300998676


In [46]:
example1 = relative_price_stragegy(start,end)

In [47]:
hold_ret,res = example1.calcu_ret(12,-0.05,-0.05,20)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5541/5541 [00:01<00:00, 3281.46it/s]


In [44]:
ff.cal_returns(hold_ret)

{'年化收益率': 0.023637137354576617,
 '年化波动率': 0.03698175048424271,
 '夏普率': 0.6391568015323665,
 '最大回撤': -0.041042231346102104,
 '收益回撤比': 0.5759223263289145,
 '胜率': 0.142,
 '盈亏比': 15.137452716560357}

In [36]:
ff.rets_all.mean()

20100104   NaN
20100105   NaN
20100106   NaN
20100107   NaN
20100108   NaN
            ..
20240429   NaN
20240430   NaN
20240506   NaN
20240507   NaN
20240508   NaN
Length: 3482, dtype: float64